In [12]:
import keras
import os
import json
import tensorflow as tf

In [13]:
# # """phase 1- delft 15"""

# data_dir=''
# event_dict=('delft15_phase2/all_event_dict_delft15.json')
# skill=os.path.join(data_dir+"skill_df_delft_15.csv")
# response=os.path.join(data_dir+"response_df_delft_15.csv")




"""phase 1- delft 14"""


# data_dir='../code/data_dkt/delft14_phase1/'
# skill_dict=json.load(open('data_dkt/delft15_phase2/skill_dict_delft_all_0_336_all.json'))
# skill=os.path.join(data_dir+"skill_df_delft_14_phase1.csv")
# response=os.path.join(data_dir+"response_df_delft_14_phase1.csv")

'phase 1- delft 14'

In [14]:

"""phase 2 """
# data_dir = "Delft15_phase2/"
# skill_dict=json.load(open(os.path.join(data_dir,"unique_events.json")))
# response = os.path.join(data_dir,"effort_response.csv")
# skill = os.path.join(data_dir,"skill.csv")


'phase 2 '

In [15]:

!ls ../code/data_dkt/delft15/

ls: cannot access ../code/data_dkt/delft15/: No such file or directory


# Changed processing file data_helper.py

In [20]:
 


import pandas as pd
import numpy as np
import json
import os

 
""" ONE HOT ENCODING OF SKILL AND RESPONSE DATA"""  


def load_data(skill_csv,response_csv,skill_json,problem_num,is_behaviour=False):
    """
    This function reads data files:
    1. skill_json:reads data from __skill_dict.json__ which maps skills to numbers 1,2..
    2. skill_csv: reads data from __skill.csv__ which is a sequence of exercise or skills attempted by a student
    3. response_csv: reads data from reponse.csv which is sequence of binary response to skill/exercise in skill.csv by each student
    4. problem_num: total number of problem events in the course, each tagged from 1 to problem_num in skill_json which a dictionary that maps events to integers
    5. is_behaviour is the boolean value which is TRUE when we want to include behavior events in our model
    """

    response_df = pd.read_csv(response_csv)
    skill_df=pd.read_csv(skill_csv)
    skill_dict=json.load(open(skill_json))

#   1. skill matrix, exclude username
    skill_matrix = skill_df.iloc[:, 1:].values
#   2. response matrix,exclude username
    response_matrix = response_df.iloc[:, 1:].values
    print('LOADED DATA:','skill_matrix .shape,response_matrix.shape')
    print(skill_matrix.shape,response_matrix.shape)
    if is_behaviour is False:
        non_problem_event_num=0
        skill_matrix[skill_matrix>problem_num]=0
    else:
        non_problem_event_num=len(skill_dict)-problem_num
    return skill_matrix,response_matrix,problem_num, non_problem_event_num


def preprocess(skill_matrix,response_matrix,problem_num=336, non_problem_event_num=291):
    """
    This function extracts the skills and responses from the loaded files excluding student ids
    :param skill_matrix: skills attempted by a student at each timestep
    :param response_matrix: responses on the exercises  by a student at each timestep
    :param problem_num: Total number of problem events in the course
    :param non_problem_num: Total number of non-problem events 
    """


#   print('CHECK:::NUMBER of NON- problem events, problem events :,', non_problem_event_num,problem_num)
#   1. skillarray
    skill_array =convert_event_to_one_hot(skill_matrix, problem_num+non_problem_event_num)
      
#  2. skill_response array
    skill_response_array = append_response_one_hot(skill_array,skill_matrix, response_matrix,problem_num)
    
#     print('skill_array.shape,response_matrix.shape,skill_response_array.shape')
#     print(skill_array.shape,response_matrix.shape,skill_response_array.shape)
    
    return skill_array, response_matrix, skill_response_array




def convert_event_to_one_hot(skill_matrix, vocab_size):
    # 1. Create one hot encode row for each integer from 0 to vocab_size
#     print(vocab_size)
    one_hot_dict = np.eye(vocab_size) 
    # 2. TO ASSIGN [000] to all out of vocab_size numbers, add last row with all zeros
    one_hot_dict = np.vstack((one_hot_dict,np.zeros(vocab_size)))
#     ADDED ONE MORE ROW SO THAT  IF WE FILL NA WITH -1 then -1-1= -2 ,
# so 2nd last row becomes the assigned value
    one_hot_dict = np.vstack((one_hot_dict,np.zeros(vocab_size)))
    # sequence length is the number timesteps in the sequence
    sequence_len = skill_matrix.shape[1] 
    # instantiation of a numpy array with all elements 0
    on_hot_sequence = np.empty((skill_matrix.shape[0],sequence_len, vocab_size))

    for row in range(skill_matrix.shape[0]):
        # set vocabulary values in skills sequence of a student equal to 1, index of skills starts at 1
        on_hot_sequence[row] = one_hot_dict [skill_matrix[row]-1] #grab 1-hot rows for integers in skill_matrix
#     print('Check encode in skill and one hot skill:', len(skill_matrix[skill_matrix!=0]),np.sum( on_hot_sequence))
    return on_hot_sequence




def append_response_one_hot(skill_array,skill_matrix, response_matrix, problem_num):
    """
   params:
       skill_matrix: 2-D matrix (student, skills)
       response_matrix:  2-D matrix (student, responses)
    with_response_vocab_size: Number of (2*problem events AND non-problem events) in the course
   returns:
       a 3d-darray with a shape like (student, sequence_len,problem_num)
   """
    # 1. Create one hot encode row for each integer from 0 to problem_num
    one_hot_dict_problems_only = np.eye(problem_num) 
    # 2. TO ASSIGN [000] to all out of vocab_size numbers
    one_hot_dict_problems_only= np.vstack((one_hot_dict_problems_only,np.zeros(problem_num)))
    #     ADDED ONE MORE ROW SO THAT  IF WE FILL NA WITH -1 then -1-1= -2 ,
# so 2nd last row becomes the assigned value
    one_hot_dict_problems_only= np.vstack((one_hot_dict_problems_only,np.zeros(problem_num))) 
    # 3. Get problem  ids of problems that are correct so that they can be assigned 1 value in one hot encode
#     all other  are 0, - ----this is why we encode are not encoding  any skill as  ----
    skill_matrix_only_correct_problems=skill_matrix*response_matrix

  
    # sequence length is the number timesteps in the sequence
    sequence_len = skill_matrix.shape[1] 
    # instantiation of a numpy array with all elements 0
    response_one_hot = np.empty((skill_matrix.shape[0],sequence_len, problem_num))
    # iterate over each student in data
    for i in range(response_matrix.shape[0]):
        # set vocabulary values in skills attempted by a student equal to 1
        # get encodes for sequences with non-zero problem
        response_one_hot[i]=one_hot_dict_problems_only[skill_matrix_only_correct_problems[i]-1]
        
#     print('Check number correct in response and one hot response:', np.sum(response_matrix),np.sum(response_one_hot))
    skill_response_array=np.concatenate((skill_array,response_one_hot),axis=2)
#     print('skill_array.shape,response_one_hot.shape,skill_response_array.shape')
#     print(skill_array.shape,response_one_hot.shape,skill_response_array.shape)
    return  skill_response_array








# DKT.PY

In [21]:
from keras.models import Model
from keras.layers import Input, Dropout, Masking, Dense, Embedding
from keras.layers import Embedding
from keras.layers.core import Flatten, Reshape
from keras.layers import LSTM
from keras.layers import merge
from keras.layers.merge import multiply
from keras.callbacks import EarlyStopping
from keras import backend as K
from theano import tensor as T
from theano import config
from theano import printing
from theano import function
from keras.layers import Lambda
import theano
import numpy as np
import pdb
from math import sqrt
from keras.callbacks import Callback
from keras.models import load_model


class DKTcustomloss():

    def __init__(self, 
                input_dim, 
                input_dim_order, 
                hidden_layer_size, 
                batch_size, 
                epochs,
                x_train=[], 
                y_train=[], 
                y_train_order=[],
                validation_split=0.0,
                validation_data=None,
                optimizer='adam',
                callbacks=None):
        """

        :param input_dim: dimension of the input at one timestamp (dimension of x_t= 2*num_skills)
        :param input_dim_order: dimension of the one-hot representation of problem to check order of occurence(=num_skills)
        :param hidden_layer_size: number of nodes in hidden layer
        :param x_train: 3D matrix of size (samples, number of timestamp/sequence length, dimension of input vec (x_t) )
        :param y_train: a matrix of responses (samples,number of timestamp/sequence length)
        :param y_train_order: shape of output equal to number of timesteps
        :param validation_split:
        :param validation_data:
        :param optimizer:
        :param callbacks:

        """
        ## input dim is the dimension of the input at one timestamp (dimension of x_t)
        self.input_dim = int(input_dim) #2* num_skills
        ## input_dim_order is the dimension of the one-hot representation of problem
        ## CHECK: order of occurence of responses should be according to timestamp
        self.input_dim_order = int(input_dim_order)#num_skills

        self.hidden_layer_size = int(hidden_layer_size)
        self.batch_size = int(batch_size)
        self.epochs = int(epochs)
        ## x_train is a 3D matrix of size (samples, number of timestamp, dimension of input vec (x_t) )
        ## in cognitive tutor # of students * # total responses * # input_dim
        self.x_train = x_train
        ## y_train is a matrix of (samples one hot representation according to problem output value at each timestamp (y_t) )
        self.y_train = y_train
        ## y_train_order is the one hot representation of problem according to timestamp starting from
        ## t=1 if training starts at t=0
        self.y_train_order = y_train_order
        # users: no of student datapoints
        self.users = np.shape(x_train)[0]
        self.validation_split = validation_split
        self.validation_data = validation_data
        self.optimizer = optimizer
        self.callbacks = callbacks
        print ("DKTnet initialization done")
        
        

        
    def LossModel(y_train,dense_out):
        dense_output= Input(batch_shape=(None, None, self.input_dim_order), name='predicted_prob')
        y_order=Input(batch_shape=(None, None, self.input_dim_order), name='skill_array')
   
        merged = multiply([dense_out, y_order])
         ## 6. Chooses the max value(which is a non zero number in merged) from the output of merged layer
        # this will reduce dimension from skill_num to 1 for each timestep

        reduced = Lambda(reduce_dim, output_shape=reduce_dim_shape)(merged)
        cce = tf.losses.softmax_cross_entropy(self.y_train,reduced)
        loss_m = Model(inputs=[dense_output,y_order], outputs=cce)
        #it's important to make this model not trainable if it has weights 
        #(you should probably set these weights manually if that's the case)    
        loss_m.trainable = False
        return loss_m

    def build_train_on_batch(self):
            ## 1. First layer for the input (x_t), creates a tensor object
            x = Input(batch_shape=(None, None, self.input_dim), name='x')

            ## 2. Mask unknown or anomalous valued timesteps in x
            # the timestep will be masked (skipped) if all values in the input tensor
            #  at that timestep are equal to mask_value
            masked = Masking(mask_value=0)(x)

            ## 3. Add a lstm layer, return sequences is True to allow output have same
            # dimension as number of timesteps in input
            lstm_out = LSTM(self.hidden_layer_size, return_sequences=True)(masked)

            ## 4. Add a fully connected layer on lstm layer, 
            ### this gives us probabilities of all events at differnt timesteps
            dense_out = Dense(self.input_dim_order, activation='sigmoid')(lstm_out)
            

            def get_probability_of_timestep_event(x):
                #Custom tensor arithmatic from backend K
                # chooses the max value from the output of previous layer
                # this will reduce dimension from one hot of skill_num to 1 for each timestep
                x = K.max(x, axis=2, keepdims=True)
                return x

        
             ## 5.1 Get problem event encoding ONLY: 
            #  ASSUMPTION:that skill dict has problem events only from keys 1:input_dim_order
            

            def get_skill_array(x):
#                 get array for  1 to t+1
                return x[:,1:,:self.input_dim_order]
            
            def get_probability_array(x):
#               get prob array for  1 to t+1 which is 0 to t index of probability array
                return x[:,:-1,:self.input_dim_order]
            def reduce_timestep_shape(input_shape):
                shape = list(input_shape)
                shape[-1]=self.input_dim_order
                return tuple(shape)

            def custom_loss(y_train,dense_out):
                '''
                using crossentropy, choose from dense_out, the probabilities corresponding to event attemted
                at that timesetep. Create a tensor object of that accepts skill_array i. e one hot encoded skill sequence
                then mask the dense_out using that skill array
                '''
                y_train=y_train[:,1:,:]

            # gets one hot encode skill without response part 0-336
                y_order=Lambda(get_skill_array)(x)
            # get probs of skills at 1 to t timesteps from  predictions by model( it predicts from 1 to t+1) 
                probabilities_for_tplus1=Lambda( get_probability_array)(dense_out)
                ## 5.2 In dense output only retain probability of event at that timestep all others 0
                merged = multiply([probabilities_for_tplus1, y_order])

                ## 6. Chooses the max value from the output of merged which is the prob
                    # this will reduce dimension from skill_num to 1 for each timestep

                reduced = Lambda(get_probability_of_timestep_event , output_shape=lambda s: (s[0], s[1],1))(merged)

                cce = K.mean(K.binary_crossentropy(y_train, reduced ))#, axis=-1)
                return cce
            
            
            def mask_prob(a):       
                condition= tf.less_equal( a, tf.constant( 0.6 ) )  
                # when con is true gets value from ist tensor if false then from 2nd tensor 
                s = tf.where (condition, tf.zeros_like(a), tf.ones_like(a)) 
                return s
                             
            def accuracy2(y_train,dense_out):
                '''this function gets then 0 to t-1 probailities from dKT 
                and compares them with 1-t actual response
                '''
                y_train=y_train[:,1:,:]
                y_order=Lambda(get_skill_array)(x) #get skill one hot encode of probs from 
                probabilities_for_tplus1=Lambda(get_probability_array)(dense_out)
            
                ## 5.2 In dense output only retain probability of event at that timestep all others 0
                merged = multiply([probabilities_for_tplus1, y_order])

                ## 6. Chooses the max value from the output of merged which is the prob
                    # this will reduce dimension from skill_num to 1 for each timestep

                reduced = Lambda(get_probability_of_timestep_event , output_shape=lambda s: (s[0], s[1],1))(merged)
             
                label=Lambda(mask_prob)(reduced)

#                 return K.mean(K.equal(y_train, K.round(reduced)))
                return K.mean(K.equal(y_train, label))


            def recall_(y_train,dense_out): 
                y_train=y_train[:,1:,:]
                y_order=Lambda(get_skill_array)(x) #get skill one hot encode of probs from 
                probabilities_for_tplus1=Lambda(get_probability_array)(dense_out)
            
                ## 5.2 In dense output only retain probability of event at that timestep all others 0
                merged = multiply([probabilities_for_tplus1, y_order])

                ## 6. Chooses the max value from the output of merged which is the prob
                    # this will reduce dimension from skill_num to 1 for each timestep

                reduced = Lambda(get_probability_of_timestep_event , output_shape=lambda s: (s[0], s[1],1))(merged)

#                 predicted_label=K.round(reduced) #<0.5=0, >0.5=1
                predicted_label=Lambda(mask_prob)(reduced)# <0.6 =0
                predicted_true_positives = K.sum(mask_prob(y_train * reduced)) #tp
    
#                 predicted_true_positives = K.sum(K.round(K.clip(y_train * reduced, 0, 1))) #tp
                possible_positives = K.sum(y_train)#tp+fn
                recall = predicted_true_positives / (possible_positives + K.epsilon())
#                 f1_score = 2*(recall * precision) / (recall + precision)              
#                 print("METRICS: f1_score,recall,precision,accuracy")
                return recall
      
            def precision_(y_train,dense_out):
                y_train=y_train[:,1:,:]
                y_order=Lambda(get_skill_array)(x) #get skill one hot encode of probs from 
                probabilities_for_tplus1=Lambda(get_probability_array)(dense_out)
            
                ## 5.2 In dense output only retain probability of event at that timestep all others 0
                merged = multiply([probabilities_for_tplus1, y_order])

                ## 6. Chooses the max value from the output of merged which is the prob
                    # this will reduce dimension from skill_num to 1 for each timestep

                reduced = Lambda(get_probability_of_timestep_event , output_shape=lambda s: (s[0], s[1],1))(merged)
#                 predicted_label=K.round(reduced) #<0.5=0, >0.5=1
                predicted_label=Lambda(mask_prob)(reduced)# <0.6 =0
#                 predicted_true_positives = K.sum(K.round(K.clip(y_train * reduced, 0, 1))) #tp
                predicted_true_positives = K.sum(mask_prob(y_train * reduced)) #tp
                predicted_positives = K.sum(K.round(K.clip(reduced, 0, 1))) #tp+fp
                precision = predicted_true_positives / (predicted_positives + K.epsilon())
#                 f1_score = 2*(recall * precision) / (recall + precision)
# #                 print("METRICS: f1_score,recall,precision,accuracy")
                return precision
         

            
  

            def accuracy_accumulated(y_train, dense_out):
                                        
                def get_cumulative_response_array(x):
                    response= x[:,:,-self.input_dim_order:]
                    for i in range(response.shape[1]):
                        while i < response.shape[1]-1:
                            response[:,i+1,:]=response[:,i+1,:]+response[:,i,:]
                    return response
                
                
   
                def cal_accuracy(x,y):
                    b= x==y
                    corr= b.astype(int)
                    return corr
                cumulative_response=Lambda(get_cumulative_response_array,output_shape=lambda s: (s[0], s[1],self.input_dim_order))(x)
                
                mask_predicted_prob = Lambda(mask_prob,output_shape=lambda s: (s[0], s[1],self.input_dim_order))(dense_out)
                
                mmse=K.mean(K.abs(mask_predicted_prob-cumulative_response))
                
                return mmse
                
                
            
            def custom_metric_accuracy(y_train,dense_out):
                '''
                     
                its gets the ratio of correct responses that got maximum accuracy in their respective timesteps
                
              
                '''
                print("using custom metric.....")
                def get_response_array(x):
                    return x[:,:,-self.input_dim_order:]
                
                actual_response=Lambda(get_response_array,output_shape=lambda s: (s[0], s[1],self.input_dim_order))(x)
                print(actual_response.get_shape(),"actual response")
                def idx_max_prob(x):
                    max_idx= K.argmax(x,axis=-1)
#                         convert to one hot, to tackle,incorrect question argmax
                    one_hot_idx=tf.one_hot(max_idx,
                                self.input_dim_order,
                                on_value=1.0,
                                off_value=0.0,
                                axis=-1)
                    return one_hot_idx
        
                max_predicted_prob = Lambda(idx_max_prob,output_shape=lambda s: (s[0], s[1],self.input_dim_order))(dense_out)
                print( max_predicted_prob.get_shape()," max_predicted_prob")
                # we use merging, we want to find how many 1-1 matchings are there
                merged_metric = multiply([actual_response,  max_predicted_prob])


                accuracy = K.mean(K.sum(merged_metric,axis=-1))
                print(accuracy.get_shape())
                return accuracy


            ## 7.Creates model object with specified input and output
                #CHANGED: OUTPUTS: TO GET probabilities at all timesteps as output.


            def custom_metric_accuracy(y_train,dense_out):
                '''
                using crossentropy, choose from dense_out, the probabilities corresponding to event attemted
                at that timesetep. Create a tensor object of that accepts skill_array i. e one hot encoded skill sequence
                then mask the dense_out using that skill array
                '''
                print("using custom metric.....")
                def get_response_array(x):
                    return x[:,:,-self.input_dim_order:]
                
                
                
                actual_response=Lambda(get_response_array,output_shape=lambda s: (s[0], s[1],self.input_dim_order))(x)
                print(actual_response.get_shape(),"actual response")
                def idx_max_prob(x):
                    max_idx= K.argmax(x,axis=-1)
#                         convert to one hot, to tackle,incorrect question argmax
                    one_hot_idx=tf.one_hot(max_idx,
                                self.input_dim_order,
                                on_value=1.0,
                                off_value=0.0,
                                axis=-1)
                    return one_hot_idx
        
                max_predicted_prob = Lambda(idx_max_prob,output_shape=lambda s: (s[0], s[1],self.input_dim_order))(dense_out)
                print( max_predicted_prob.get_shape()," max_predicted_prob")
                # we use merging, we want to find how many 1-1 matchings are there
                merged_metric = multiply([actual_response,  max_predicted_prob])


                accuracy = K.mean(K.sum(merged_metric,axis=-1))
                print(accuracy.get_shape())
                return accuracy


            ## 7.Creates model object with specified input and output
                #CHANGED: OUTPUTS: TO GET probabilities at all timesteps as output.

            self.model = Model(inputs=x, outputs=dense_out)

            ## 8. Compile model by assigning loss function for backpropagtion
            self.model.compile(optimizer=self.optimizer,
                               loss=custom_loss,
                               metrics=[recall_,precision_,accuracy2])

            print('Summary of the model')
            self.model.summary()
   

    def train_on_batch(self, x_train,y_train,y_train_order):
# y_train_order: not used
        self.model.train_on_batch(x_train, y_train)




    def test_on_batch(self, x_val, y_val,y_train_order):
        
        # y_train_order: not used
        """
       Test the model on a single batch of samples
       :return: Scalar test loss (if the model has a single output and no metrics) or list of scalars (if the model has multiple outputs and/or metrics).
       The attribute model.metrics_names will give you the display labels for the scalar outputs.
       """
#         print(self.model.metrics_names)
        return self.model.test_on_batch(x_val, y_val)

    def predict(self, x_val,y_train_order):
#              # y_train_order: not used
        return self.model.predict_on_batch(x_val)

    def save(self):
             # Save the weights
        print('saved model','phase1_model_weights.h5','model_architecture_phase1.json')
        # Save the weights
        self.model.save_weights('model_weights.h5')

        # Save the model architecture
        with open('model_architecture_phase1.json', 'w') as f:
            f.write(self.model.to_json())





# train.py

In [22]:
import pandas as pd
import numpy as np
import json
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import argparse

# from DKT import DKTnet
# from data_helper import load_data, one_hot, preprocess

def get_callbacks():
    '''
    Some callback functions that you may find useful.
    Please refer to https://keras.io/callbacks/ for more detailed explaination
    '''
    checkpoint = ModelCheckpoint('my_model', 
                                 monitor='val_loss',
                                 verbose=2, 
                                 save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', 
                                   patience=2)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=0, min_lr=1e-4)
    return [checkpoint, early_stopping, reduce_lr]



def batch_generator(skill_array, response_array, batch_size=64, shuffle=True):
    """
    return: batches of data from the original data set for training
    
    """
    skill_array, response_array, skill_response_array=preprocess(skill_array, response_array)
    sample_num = skill_array.shape[0]
    if shuffle:
        shuffled_indices = np.random.permutation(sample_num)
        skill_array = skill_array.copy()[shuffled_indices]
        response_array = response_array.copy()[shuffled_indices]
        skill_response_array = skill_response_array.copy()[shuffled_indices]
        print('Training set shuffled')
    for ndx in range(0, sample_num, batch_size):
        skill_array_batch = skill_array[ndx:min(ndx+batch_size, sample_num)]
        response_array_batch = response_array[ndx:min(ndx+batch_size, sample_num)]
        skill_response_array_batch = skill_response_array[ndx:min(ndx+batch_size, sample_num)]
        yield skill_response_array_batch, response_array_batch, skill_array_batch

def create_validation_data(skill_array, response_array,size=0.2):
    """
    return: split of data from the original data set for testing
    """

    sample_num = skill_array.shape[0]
    shuffled_indices = np.random.permutation(sample_num)
    skill_array = skill_array.copy()[shuffled_indices]
    response_array = response_array.copy()[shuffled_indices]
#     skill_response_array = skill_response_array.copy()[shuffled_indices]
    print('Data set shuffled, preparing for split')
    split_index=int(size*sample_num)
    train_index=sample_num-split_index
    print('train:', train_index, "test:",split_index)
    skill_array_train = skill_array[0:train_index]
    response_array_train = response_array[0:train_index]
#     skill_response_array_train = skill_response_array[0:train_index]
    
    skill_array_test = skill_array[train_index:]
    response_array_test = response_array[train_index:]
#     skill_response_array_test = skill_response_array[train_index:]
    
    return skill_array_train, response_array_train,skill_array_test,response_array_test


def train_on_batch(skill_array, response_array,custom_loss=False,):
    """This function creates a DKT MODEL object using DKT.py and
    trains it using the batched data"""

    input_dim = 627+336  #2* num_skills
#     input_dim_order = skill_array.shape[-1] #num_skills
    input_dim_order = 336
    hidden_layer_size = 40
    batch_size = 64
    epochs = 5
    print("batch size==",batch_size)
    print("epoch size=", epochs)
    print("input_dim_order",input_dim_order)
    print("input_dim",input_dim)
    

    '''
    parameters like batch_size, epochs x_train, y_train, y_train_order and validations are not used
    if you are doing a training by batch
    '''

    dkt = DKTcustomloss(input_dim, 
                input_dim_order, 
                hidden_layer_size, 
                batch_size, 
                epochs,
#                     x_train=skill_response_array, 
#                     y_train=response_array[:, :, np.newaxis], 
#                     y_train_order=skill_array,
                validation_split=0.2,
                validation_data=0.2,
                optimizer='adam',
                callbacks=None)

        
        
    print('x_train.shape,y_train.shape')
    print(skill_array.shape,response_array.shape)
   
    dkt.build_train_on_batch()
    

    '''
    For simplification, we are over fitting on the training set here.
    In your model, you should do a train-test split or cross-validation which can be found in sklearn package.
    '''
    skill_array_train, response_array_train,skill_array_test,response_array_test=create_validation_data(skill_array, response_array,size=dkt.validation_split)

    df_metrics=pd.DataFrame(columns=['epoch','train_loss','train_recall','train_precision','train_accuracy','test_loss','test_recall','test_precision','test_accuracy'])
    for e in range(epochs):
        print('***Epoch', e+1, 'starts****')
        j = 0
        train_loss=0
        train_recall=0
        train_precision=0
        train_accuracy=0
        test_loss=0
        test_recall=0
        test_precision=0
        test_accuracy=0
#         test_accuracy=np.zeros(3)
        total_iteration_num = 1 + (skill_array.shape[0] - 1) // batch_size
        for skill_response_array_batch, response_array_batch, skill_array_batch in batch_generator(skill_array_train, response_array_train,  batch_size=batch_size):
            dkt.train_on_batch(skill_response_array_batch,response_array_batch[:, :, np.newaxis],  skill_array_batch)
            loss,recall,precision,accuracy=dkt.test_on_batch(skill_response_array_batch,response_array_batch[:, :, np.newaxis],  skill_array_batch)
            train_loss+=loss
            train_recall+=recall
            train_precision+=precision
            train_accuracy+=accuracy
            j += 1
        train_metrics=[train_loss/j,train_recall/j,train_precision/j,train_accuracy/j]
        print('Evalutaion training data result,loss,recall,precision,accuracy', train_metrics)
        print("iter: {}/{} done".format(j, total_iteration_num))
        i=0
        for sr, r, s in batch_generator(skill_array_test, response_array_test,  batch_size=batch_size):
            loss,recall,precision,accuracy=dkt.test_on_batch(sr,r[:, :, np.newaxis], s)
            test_loss+=loss
            test_recall+=recall
            test_precision+=precision
            test_accuracy+=accuracy
            i+=1
        test_metrics=[test_loss/i,test_recall/i,test_precision/i,test_accuracy/i]
        print('Evalutaion validation data result, loss, f1_score,recall,precision,accuracy',test_metrics)
        all_metrics=[e]+train_metrics+test_metrics
        df_metrics.loc[e]=all_metrics
#     df_metrics.to_csv('phase_2_metrics.csv')

    dkt.save()
    print(df_metrics)
    return dkt



In [24]:

# """phase 2- delft 15"""

data_dir='code/delft15_phase2/'
event_dict=(data_dir+'all_event_dict_delft15.json')
skill=os.path.join(data_dir+"train_event_df_delft_15.csv")
response=os.path.join(data_dir+"train_response_df_delft_15.csv")


# subset for testing

skill_matrix,response_array,problem_num, non_problem_event_num=load_data(skill,response,event_dict,problem_num=336,is_behaviour=True)



# skill_matrix=skill_matrix[:500,:]
# response_array=response_array[:500,:]

# # load preprocess in each batch, put in train_batch fynction
# print('Number of non-problem events:', non_problem_event_num)
# skill_array, response_array, skill_response_array=preprocess(skill_matrix,response_array,problem_num, non_problem_event_num)

model=train_on_batch(skill_matrix,response_array,custom_loss=True)


LOADED DATA: skill_matrix .shape,response_matrix.shape
(8242, 100) (8242, 100)
batch size== 64
epoch size= 5
input_dim_order 336
input_dim 963
DKTnet initialization done
x_train.shape,y_train.shape
(8242, 100) (8242, 100)
Summary of the model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
x (InputLayer)               (None, None, 963)         0         
_________________________________________________________________
masking_4 (Masking)          (None, None, 963)         0         
_________________________________________________________________
lstm_4 (LSTM)                (None, None, 40)          160640    
_________________________________________________________________
dense_4 (Dense)              (None, None, 336)         13776     
Total params: 174,416
Trainable params: 174,416
Non-trainable params: 0
_________________________________________________________________
Data set shuffled, prepar

In [25]:
skill_test=os.path.join(data_dir+"test_event_df_delft_15.csv")
response_test=os.path.join(data_dir+"test_response_df_delft_15.csv")
# response_test = pd.read_csv(response)
# skill_test=pd.read_csv(skill)

# subset for testing
skill_test,response_test,problem_num, non_problem_event_num=load_data(skill_test,response_test,event_dict,problem_num=336,is_behaviour=False)
i=0
test_loss=0
test_recall=0
test_precision=0
test_accuracy=0
for sr, r, s in batch_generator(skill_test, response_test,  batch_size=100):
    loss,recall,precision,accuracy=model.test_on_batch(sr,r[:, :, np.newaxis], s)
    test_loss+=loss
    test_recall+=recall
    test_precision+=precision
    test_accuracy+=accuracy
    i+=1
test_metrics=[test_loss/i,test_recall/i,test_precision/i,test_accuracy/i]
print('Evalutaion test data result, loss,recall,precision,accuracy',test_metrics)


LOADED DATA: skill_matrix .shape,response_matrix.shape
(1000, 100) (1000, 100)
Training set shuffled
Evalutaion test data result, loss,recall,precision,accuracy [0.1525301605463028, 0.5937254548072814, 0.6363957464694977, 0.9219888210296631]


In [26]:
# train_on_batch(skill_array,response_array, skill_response_array,custom_loss=True)

In [ ]:
# train_on_batch(skill_array,response_array, skill_response_array)

In [ ]:
k=[1,2]
l=np.array(k)
s=l+l
s/2

In [2]:
import numpy as np
s=[]
s=np.zeros(3)
s

array([0., 0., 0.])

In [3]:
k=np.array([0,0,0,0,10,15,6,7]).reshape(2,2,2)
k

array([[[ 0,  0],
        [ 0,  0]],

       [[10, 15],
        [ 6,  7]]])

In [11]:
import tensorflow as tf
sess = tf.InteractiveSession()
s=(tf.argmax(k,axis=-1))
s.eval().shape
o=tf.one_hot(
    s,
    4,
    on_value=3,
    off_value=0,
    axis=-1,
    dtype=None
)


/usr/local/lib/python3.4/dist-packages/tensorflow/python/client/session.py:1711: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [ ]:
# c = tf.constant([[1.0, 2.0], [3.0, 4.0]])
# s = tf.less_equal(c, tf.constant(2,2))
# # s=tf.where(s,c,tf.constant(-1))



a = tf.constant( [0.1,0.9,0.6,0.5] )       
con= tf.less_equal( a, tf.constant( 0.6 ) )  
# when con is true gets value from ist tensor if false then from 2nd tensor 
s = tf.where (k, tf.zeros_like(a), tf.ones_like(a)) 

s.eval()

In [13]:
def mask_prob(x):
    cond = tf.less_equal(x, tf.constant(3))
    return tf.where(cond, x, tf.constant(-1))
mask_prob(o).eval()

ValueError: Shapes must be equal rank, but are 3 and 0 for 'Select_3' (op: 'Select') with input shapes: [2,2,4], [2,2,4], [].

In [ ]:
K.mean(K.sum(o,axis=-1)).eval()

In [ ]:
K.sum(o,axis=-1).eval()

In [ ]:
s=[0,0]
s+=[1,2]
s